In [2]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

data_path = Path("../../data/")   # full path to the HEF folder

train_df = pd.read_csv(data_path / "mimic_train_HEF.csv", low_memory=False)
test_df  = pd.read_csv(data_path / "mimic_test_HEF.csv",  low_memory=False)

train_df.shape, test_df.shape

((20885, 44), (5221, 39))

drop all unnecesarry colums (maybe later keep DOB for age)
SPo2 ook nog is nazien of we die er wel best uitlaten 

In [4]:
import pandas as pd
import numpy as np


def summarize_dataframe(df, max_samples=10):
    summary = []

    for col in df.columns:
        dtype = df[col].dtype
        series = df[col]

        n_missing = series.isna().sum()
        n_unique = series.nunique(dropna=True)

        if np.issubdtype(dtype, np.number):
            # Numeric summary
            col_min = series.min()
            col_max = series.max()
            col_mean = series.mean()
            col_std = series.std()

            # Example numeric values (non-missing)
            examples = series.dropna().sample(
                min(max_samples, len(series.dropna())), 
                random_state=42
            ).round(2).astype(str).tolist()

            example_values = ", ".join(examples)

        else:
            # Categorical summary
            col_min = col_max = col_mean = col_std = None

            # Get sample categories WITHOUT missing values
            unique_vals = (
                series.dropna()
                .unique()[:max_samples]
            )  # already excludes NaN

            example_values = ", ".join(map(str, unique_vals))

        summary.append({
            "column": col,
            "dtype": str(dtype),
            "missing": n_missing,
            "unique": n_unique,
            "min": col_min,
            "max": col_max,
            "mean": col_mean,
            "std": col_std,
            "sample_values": example_values
        })

    return pd.DataFrame(summary)


# Run on your cleaned X dataset (after dropping leakage + IDs)
summary_df = summarize_dataframe(train_df)
summary_df


,column,dtype,missing,unique,min,max,mean,std,sample_values
0,HOSPITAL_EXPIRE_FLAG,int64,0,2,0.000000,1.000000,0.112282,0.315720,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0"
1,subject_id,int64,0,16317,23.000000,99999.000000,58950.496098,25299.439535,"63519, 74795, 54587, 19296, 85539, 69123, 5921..."
2,hadm_id,int64,0,19749,100001.000000,199999.000000,150082.402298,28898.479845,"170597, 102641, 102683, 186066, 179291, 159666..."
3,icustay_id,int64,0,20885,200001.000000,299998.000000,250202.495523,28909.806302,"211145, 215021, 218147, 231806, 202318, 201748..."
4,HeartRate_Min,float64,2187,131,2.000000,141.000000,69.705904,14.869840,"71.0, 72.0, 66.0, 74.0, 62.0, 68.0, 66.0, 73.0..."
5,HeartRate_Max,float64,2187,164,39.000000,280.000000,105.239801,20.922613,"98.0, 134.0, 107.0, 103.0, 77.0, 99.0, 98.0, 1..."
6,HeartRate_Mean,float64,2187,14091,34.714286,163.875000,85.180250,15.318208,"82.29, 96.17, 80.58, 91.76, 69.0, 81.6, 80.07,..."
7,SysBP_Min,float64,2208,154,5.000000,181.000000,91.110564,17.532534,"98.0, 100.0, 101.0, 77.0, 97.0, 83.0, 80.0, 60..."
8,SysBP_Max,float64,2208,190,46.000000,323.000000,150.725920,23.833793,"130.0, 165.0, 157.0, 131.0, 124.0, 116.0, 135...."
9,SysBP_Mean,float64,2208,13779,46.000000,202.172414,119.145423,16.701503,"111.5, 141.17, 121.62, 91.28, 112.32, 99.88, 9..."


In [ ]:
# --- 1. Columns to drop (from our earlier analysis)
drop_cols = ['LOS', 'subject_id', 'hadm_id', 'icustay_id', 'ADMITTIME','DISCHTIME', 'DEATHTIME', 'DOD', 'DOB', 'Diff', 'SpO2_Max']

# --- 2. Target column
target_col = 'HOSPITAL_EXPIRE_FLAG'

# --- 3. Create X (features) and y (target)
X = train_df.drop(columns=drop_cols + [target_col])
y = train_df[target_col]

# --- 4. Also drop the same columns from the test set
X_test = test_df.drop(columns=drop_cols, errors='ignore')

# --- 5. Confirm same feature columns in train/test
print("Train shape:", X.shape)
print("Test shape:", X_test.shape)
print("Same columns in train/test:", set(X.columns) == set(X_test.columns))


Train shape: (20885, 32)
Test shape: (5221, 32)
Same columns in train/test: True


In [21]:
import numpy as np

numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

print("Numeric columns:", len(numeric_cols))
print("Categorical columns:", len(categorical_cols))


Numeric columns: 23
Categorical columns: 9


## Imputing missing data

In [22]:
X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].median())
print("Remaining missing values:", X.isna().sum().sum())
# remaining is for the categorical colum 

Remaining missing values: 722


In [23]:
for col in categorical_cols:
    X[col] = X[col].fillna(X[col].mode()[0])
print("Remaining missing values after imputing categorical columns:", X.isna().sum().sum())

Remaining missing values after imputing categorical columns: 0


## outliers